In [ ]:
# Descargar herramientas de c++https://visualstudio.microsoft.com/es/visual-cpp-build-tools/
# Descargar el modelo desde: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
import cv2
import dlib
from deepface import DeepFace
import numpy as np
import random

def overlay_image(background, overlay, pos_x, pos_y):
    """
    Overlay an image with an alpha channel over another image at a specific position,
    ensuring it does not go out of bounds of the frame.
    """
    h, w = overlay.shape[:2]

    # Crop the overlay if it goes out of the frame bounds
    if pos_y < 0:
        overlay = overlay[-pos_y:, :]
        pos_y = 0
    if pos_x < 0:
        overlay = overlay[:, -pos_x:]
        pos_x = 0
    if pos_y + h > background.shape[0]:
        overlay = overlay[:background.shape[0] - pos_y, :]
    if pos_x + w > background.shape[1]:
        overlay = overlay[:, :background.shape[1] - pos_x]

    # Apply the overlay using the alpha channel
    alpha_overlay = overlay[:, :, 3] / 255.0  # Alpha channel for transparency
    for c in range(0, 3):
        background[pos_y:pos_y + overlay.shape[0], pos_x:pos_x + overlay.shape[1], c] = \
            (1 - alpha_overlay) * background[pos_y:pos_y + overlay.shape[0], pos_x:pos_x + overlay.shape[1], c] + \
            alpha_overlay * overlay[:, :, c]
    return background


def load_image(path):
    return cv2.imread(path, cv2.IMREAD_UNCHANGED)  # Complete path to Chopper's hat

def zoro_scar_filter(frame, landmarks):
    # Posiciones clave para la cicatriz y el ojo
    forehead = (landmarks.part(19).x, landmarks.part(19).y)
    cheek = (landmarks.part(31).x, landmarks.part(31).y)
    eye_top = landmarks.part(37).y  # Punto superior del ojo izquierdo
    eye_bottom = landmarks.part(41).y  # Punto inferior del ojo izquierdo

    scar_color = (50, 50, 52)
    scar_thickness = 2
    num_lines = 4
    
    # Segmento superor de la cicatriz (justo encima del ojo) 
    for i in range(num_lines):
        offset_x = random.randint(-1, 1)  # Variación horizontal para simular textura de cicatriz
        start_point = (forehead[0] + offset_x, forehead[1] + i * 5)
        end_point = (forehead[0] + offset_x, eye_top - 5 - i * 2)

        cv2.line(frame, start_point, end_point, scar_color, scar_thickness)

    # Segmento inferior de la cicatriz (justo bajo el ojo)
    for i in range(num_lines):
        offset_x = random.randint(-1, 1)
        start_point = (forehead[0] + offset_x, eye_bottom + 5 + i * 3)
        end_point = (forehead[0] + offset_x, cheek[1] - i * 2)

        cv2.line(frame, start_point, end_point, scar_color, scar_thickness)
    

def choper_hat_filter(frame, landmarks):
    chopper_hat = load_image("chopper_hat.png")

    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
    left_temple = (landmarks.part(17).x, landmarks.part(17).y)  # Left temple point
    right_temple = (landmarks.part(26).x, landmarks.part(26).y)  # Right temple point
    chin = (landmarks.part(8).x, landmarks.part(8).y)

    head_width = int(2.5 * (right_temple[0] - left_temple[0]))  # Increased width for better coverage
    head_height = int(1.5 * abs(chin[1] - left_eye[1]))   # Adjusted height for appropriate look
    chopper_hat = cv2.resize(chopper_hat, (head_width, head_height))
    
    # Position the hat: centered over the forehead and head
    pos_x = left_temple[0] - (head_width *2 // 7)  # Center horizontally
    pos_y = left_eye[1] - int(head_height * 0.95)  # Position higher to cover the top of the head

    # Overlay the hat on the image
    overlay_image(frame, chopper_hat, pos_x, pos_y)


def afro_brook_filter(frame, landmarks):
    afro_brook = load_image("afro_brook.png")
    
    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
    left_temple = (landmarks.part(17).x, landmarks.part(17).y)  
    right_temple = (landmarks.part(26).x, landmarks.part(26).y) 
    chin = (landmarks.part(8).x, landmarks.part(8).y)

    head_width = int(2.5 * (right_temple[0] - left_temple[0])) 
    head_height = int(1.5 * abs(chin[1] - left_eye[1]))  
    afro_brook = cv2.resize(afro_brook, (int(head_width*1.4), int(head_height*1.8)))
    

    pos_x = left_temple[0] - (head_width *2 // 4)  
    pos_y = left_eye[1] - int(head_height * 1.3)  

    overlay_image(frame, afro_brook, pos_x, pos_y)


def luffy_hat_filter(frame, landmarks):
    luffy_hat = load_image("luffy_hat.png") 

    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
    left_temple = (landmarks.part(17).x, landmarks.part(17).y)  
    right_temple = (landmarks.part(26).x, landmarks.part(26).y) 
    chin = (landmarks.part(8).x, landmarks.part(8).y)

    head_width = int(3.0 * (right_temple[0] - left_temple[0])) 
    head_height = int(1.8 * abs(chin[1] - left_eye[1]))
    luffy_hat = cv2.resize(luffy_hat, (head_width, head_height))
    
    pos_x = left_temple[0] - (head_width * 2 // 6)  
    pos_y = left_eye[1] - int(head_height * 0.8)  

    overlay_image(frame, luffy_hat, pos_x, pos_y)

def luffy_scar_filter(frame, landmarks):
    luffy_scar = load_image("luffy_scar.png") 

    left_eye_bottom = landmarks.part(41) 

    scar_width = int(0.3 * (landmarks.part(16).x - landmarks.part(0).x))  # Proporcional al ancho de la cara
    scar_height = int(scar_width * luffy_scar.shape[0] / luffy_scar.shape[1])  # Mantener la proporción de la imagen
    luffy_scar = cv2.resize(luffy_scar, (scar_width, scar_height))
    
    pos_x = left_eye_bottom.x - scar_width // 2
    pos_y = left_eye_bottom.y + 2

    overlay_image(frame, luffy_scar, pos_x, pos_y)

def luffy_filter(frame, landmarks):
    
    luffy_hat_filter(frame, landmarks)
    
    luffy_scar_filter(frame, landmarks)

function_list = [ afro_brook_filter, zoro_scar_filter, choper_hat_filter, luffy_filter]

predictor = dlib.shape_predictor( "shape_predictor_68_face_landmarks.dat")

video_capture = cv2.VideoCapture(0)
index = 0  # Índice inicial para function_list

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    try:
        # Extraer las caras usando DeepFace
        faces = DeepFace.extract_faces(frame, detector_backend='yolov8')
        
        for face in faces:
            x, y = face['facial_area']['x'], face['facial_area']['y']
            w, h = face['facial_area']['w'], face['facial_area']['h']
            dlib_rect = dlib.rectangle(left=x, top=y, right=x + w, bottom=y + h)
            
            landmarks = predictor(gray, dlib_rect)
            
            # Ejecutar la función actual en function_list basada en 'index'
            function_list[index](frame, landmarks)

    except Exception as e:
        print(e)

    # Mostrar la imagen con el filtro aplicado
    cv2.imshow("One Piece Filter", frame)
    
    # Leer la tecla presionada
    key = cv2.waitKey(5) & 0xFF

    if key == ord("q"):
        # Decrementar el índice y asegurarse de que esté en rango
        index = (index - 1) % len(function_list)
    elif key == ord("e"):
        # Incrementar el índice y asegurarse de que esté en rango
        index = (index + 1) % len(function_list)
    elif key == 27:  # Código ASCII para 'Esc'
        # Salir del bucle y cerrar la ventana
        break

video_capture.release()
cv2.destroyAllWindows()
